In [ ]:
import tensorflow as tf
from tensorflow.keras import layers, models, regularizers
from tensorflow.keras.datasets import fashion_mnist
from tensorflow.keras.utils import to_categorical
from sklearn.model_selection import KFold
import numpy as np
import matplotlib.pyplot as plt

# Cargar el dataset Fashion MNIST
(train_images, train_labels), (test_images, test_labels) = fashion_mnist.load_data()

# Preprocesamiento
train_images = train_images.reshape((60000, 28 * 28)).astype('float32') / 255
test_images = test_images.reshape((10000, 28 * 28)).astype('float32') / 255

# Categorización de las etiquetas
train_labels = to_categorical(train_labels)
test_labels = to_categorical(test_labels)

# Parámetros de Cross Validation
kfold = KFold(n_splits=5, shuffle=True)

# Almacenar resultados
accuracy_per_fold = []
loss_per_fold = []

# CROSS VALIDATION
fold_no = 1
for train_index, val_index in kfold.split(train_images, train_labels):
    print(f"Fold {fold_no}")

    # Dividir los datos en conjuntos de entrenamiento y validación
    X_train, X_val = train_images[train_index], train_images[val_index]
    y_train, y_val = train_labels[train_index], train_labels[val_index]

    # Construir el modelo MLP con regularización y Dropout
    model = models.Sequential()
    model.add(layers.Dense(512, activation='relu', input_shape=(28 * 28,), kernel_regularizer=regularizers.l2(0.001)))
    model.add(layers.Dropout(0.3))
    model.add(layers.Dense(256, activation='relu', kernel_regularizer=regularizers.l2(0.001)))
    model.add(layers.Dropout(0.3))
    model.add(layers.Dense(128, activation='relu'))
    model.add(layers.Dropout(0.3))
    model.add(layers.Dense(10, activation='softmax'))

    # Compilar el modelo
    model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.0001),
                  loss='categorical_crossentropy',
                  metrics=['accuracy'])

    # Entrenar el modelo en los datos de entrenamiento del pliegue
    history = model.fit(X_train, y_train,
                        epochs=20,
                        batch_size=128,
                        validation_data=(X_val, y_val),
                        verbose=1)

    # Evaluar el modelo en los datos de validación del pliegue
    scores = model.evaluate(X_val, y_val, verbose=0)
    print(f"Score for fold {fold_no}: {model.metrics_names[0]} of {scores[0]}; {model.metrics_names[1]} of {scores[1]}")
    accuracy_per_fold.append(scores[1])
    loss_per_fold.append(scores[0])

    fold_no += 1

# Resultados finales del cross-validation
print('Resultados por pliegue:')
for i in range(0, len(accuracy_per_fold)):
    print(f"> Fold {i+1} - Pérdida: {loss_per_fold[i]} - Precisión: {accuracy_per_fold[i]}")
print(f"Precisión promedio: {np.mean(accuracy_per_fold)}")
print(f"Pérdida promedio: {np.mean(loss_per_fold)}")

# Evaluar el modelo final en el conjunto de test
test_loss, test_acc = model.evaluate(test_images, test_labels)
print(f"Precisión en el conjunto de test: {test_acc}")

# Crear gráfico para mostrar la precisión y pérdida de cada fold
folds = np.arange(1, len(accuracy_per_fold) + 1)

# Gráfico de precisión
plt.figure(figsize=(12, 6))

plt.subplot(1, 2, 1)
plt.plot(folds, accuracy_per_fold, marker='o', linestyle='-', color='blue', label='Accuracy')
plt.title('Accuracy per Fold')
plt.xlabel('Fold')
plt.ylabel('Accuracy')
plt.xticks(folds)
plt.grid(True)
plt.legend()

# Gráfico de pérdida
plt.subplot(1, 2, 2)
plt.plot(folds, loss_per_fold, marker='o', linestyle='-', color='red', label='Loss')
plt.title('Loss per Fold')
plt.xlabel('Fold')
plt.ylabel('Loss')
plt.xticks(folds)
plt.grid(True)
plt.legend()

# Mostrar el gráfico
plt.tight_layout()
plt.show()

Fold 1
Epoch 1/20
375/375 ━━━━━━━━━━━━━━━━━━━━ 9s 21ms/step - accuracy: 0.4953 - loss: 2.3689 - val_accuracy: 0.8025 - val_loss: 1.3597
Epoch 2/20
375/375 ━━━━━━━━━━━━━━━━━━━━ 6s 16ms/step - accuracy: 0.7743 - loss: 1.4285 - val_accuracy: 0.8328 - val_loss: 1.1688
Epoch 3/20
375/375 ━━━━━━━━━━━━━━━━━━━━ 11s 18ms/step - accuracy: 0.8127 - loss: 1.2279 - val_accuracy: 0.8427 - val_loss: 1.0609
Epoch 4/20
375/375 ━━━━━━━━━━━━━━━━━━━━ 11s 20ms/step - accuracy: 0.8281 - loss: 1.1040 - val_accuracy: 0.8528 - val_loss: 0.9754
Epoch 5/20
375/375 ━━━━━━━━━━━━━━━━━━━━ 9s 17ms/step - accuracy: 0.8402 - loss: 1.0099 - val_accuracy: 0.8594 - val_loss: 0.9046
Epoch 6/20
375/375 ━━━━━━━━━━━━━━━━━━━━ 10s 16ms/step - accuracy: 0.8500 - loss: 0.9379 - val_accuracy: 0.8619 - val_loss: 0.8551
Epoch 7/20
375/375 ━━━━━━━━━━━━━━━━━━━━ 10s 16ms/step - accuracy: 0.8552 - loss: 0.8791 - val_accuracy: 0.8650 - val_loss: 0.8048
Epoch 8/20
375/375 ━━━━━━━━━━━━━━━━━━━━ 11s 17ms/step - accuracy: 0.8588 - loss: 0.831